In [1]:
# Load libraries
import time
import ee
import geemap
import os
import numpy
import pandas as pd
import ee.mapclient
import pyarrow as pa

In [ ]:
# Connect to GEE
ee.Authenticate()
ee.Initialize(project = 'clearing-blackwaters')

# Load in helpers from Gardner et al analysis 
exec(open("rhea_GEE_pull_functions_rivers.py").read())

# load in Landsat datasets
l9 = ee.ImageCollection("LANDSAT/LC09/C02/T1_L2")
l8 = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2")
l7 = ee.ImageCollection("LANDSAT/LE07/C02/T1_L2")
l5 = ee.ImageCollection("LANDSAT/LT05/C02/T1_L2")

# Identify collection for use in sourced functions.
collection = 'L2'

# Standardize band names between the various collections and aggregate them into one image collection
bn89 = ['SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7', 'QA_PIXEL']
bn57 = ['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B7', 'QA_PIXEL']
bns = ['Blue', 'Green', 'Red', 'Nir', 'Swir1', 'Swir2', 'qa']
  
ls5 = l5.select(bn57, bns)
ls7 = l7.select(bn57, bns)
ls8 = l8.select(bn89, bns)
ls9 = l9.select(bn89, bns)

# Set cloud cover threshold
ls = ee.ImageCollection(ls5.merge(ls7).merge(ls9)).filter(ee.Filter.lt('CLOUD_COVER', 50))

# Load in flowline of interest 
all_flowlines = ee.data.listAssets({"parent": "projects/clearing-blackwaters/assets"})

# Get flowline IDs
flowlinesID = []
for x in range(0, len(all_flowlines['assets'])):
    flowline_id = all_flowlines['assets'][x]['id']
    flowlinesID.append(flowline_id)

In [ ]:
# Loops through split dates (every year 1984-2024) downloads by DSWE

errors = list()

for i in range(0, len(flowlinesID)):

    # Select individual flowline
    flowline = ee.FeatureCollection(flowlinesID[i])

    # Get all comids 
    flowline_comids = flowline.aggregate_array('comid').getInfo()
    
    # Loop through comids
    for x in range(0, len(flowline_comids)):
        
        # get individual comid 
        flowline_fc = flowline.filter(ee.Filter.eq('comid', flowline_comids[x]))
        
        # Function to buffer each feature
        def buffer_feature(feature):
            return feature.buffer(100, 10)
 
        # Map the buffer function over the collection
        this_line = flowline_fc.map(buffer_feature)

        # Apply correction factor for Collection 1 to Collection 2
        def apply_scale_factors(image):
            optical_bands = image.select(['Blue', 'Green', 'Red', 'Nir', 'Swir1', 'Swir2']).multiply(0.0000275).add(-0.2)
            return image.addBands(optical_bands, None, True)
        
        # Cloud and shadow mask
        def qa_mask(image): 
            qa = image.select('qa').bitwiseAnd(int('11111', 2)).eq(0)
            return image.updateMask(qa)
     
        # Loop through every year
        count = 1
        for y in range (1984, 2024): 

            # Loop through every 3 months
            timerange = [f"{y}-06-01", f"{y}-08-31", f"{y}-09-01", f"{y}-11-31", f"{y}-12-01", f"{y+1}-2-27", f"{y+1}-02-28", f"{y}-05-31", ]
            
            for z in range (0, 8, 2): 

                # Filter landsat image to comid 
                ls_masked = ls.filterDate(timerange[z], timerange[z+1]).filterBounds(this_line).map(apply_scale_factors).map(qa_mask)

                # Apply DSWE algorithm 
                ls_masked_dswe = ls_masked.map(Dswe)
                
                try: 
                    # Pull data for DSWE, filter only for values above 0 and max 2
                    data_dswe = ls_masked_dswe.getRegion(this_line, 30)
                    data_dswe_info = data_dswe.getInfo()
                    data_dswe_filtered = list()
                    data_filtered_ids = list()
                    for d in data_dswe_info: 
                        if (d[4] != None) and (((isinstance(d[4], float) or isinstance(d[4], int)) and (0 < d[4] <= 2.0)) or (isinstance(d[4], str))): 
                            data_dswe_filtered.append(d)
                            data_filtered_ids.append((d[0], d[1], d[2]))
                    keys_dswe = ee.List(data_dswe.get(0)).getInfo()

                    # Change save directory as needed
                    save_path_dswe = "D:/dwz6/Documents/landsat_data_dswe/" + str(flowline_comids[x]) + " (" + str(count) + ").feather"
                    
                    pd.DataFrame(data_dswe_filtered[1:len(data_dswe_filtered)],
                                columns = keys_dswe).to_feather(save_path_dswe)
                    
                    # Pull data for image based on the downloaded DSWE files
                    data = ls_masked.getRegion(this_line, 30)
                    data_info = data.getInfo()
                    data_filtered = list()
                    for y in data_info: 
                        if (y[0], y[1], y[2]) in data_filtered_ids: 
                            data_filtered.append(y)
                    keys = ee.List(data.get(0)).getInfo()

                    # Change save directory as needed
                    save_path = "D:/dwz6/Documents/landsat_data/" + str(flowline_comids[x]) + " (" + str(count) + ").feather"
                    
                    pd.DataFrame(data_filtered[1:len(data_filtered)],
                                columns = keys).to_feather(save_path)
                except: 
                    # Print errors and add to a list
                    error_comid = "comid: " + str(flowline_comids[x]) + " (" + str(count) + ") (asset: " + str(flowlinesID[i]) + ")"
                    print("Error:", error_comid)
                    errors.append(error_comid)
                
                count += 1